In [1]:
import pandas as pd
import numpy as np

Some Functions

In [2]:
#Returns the unit vector of the vector.
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

#Returns the angle in radians between vectors 'v1' and 'v2'
def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

#rotates a vector
def affineTransform(vector, angle, position):
    rotation_matrix = np.array([[np.cos(angle), -1 * np.sin(angle)],
                                [np.sin(angle),      np.cos(angle)]])

    return np.matmul(np.linalg.pinv(rotation_matrix), vector - position)

#changes points to match the inteface of the movement component
def pointsInterface(points, direction, position):
    vectors = []

    #x and y are mirrored in unreal engine
    if direction['y'] < 0:
        right_rotation = False
    else:
        right_rotation = True

    theta = angle_between(np.array([0,1]), np.array([direction['y'], direction['x']]))
    
    if right_rotation:
       theta *= -1

    for point in points:
        x = point['x']
        y = point['y']

        mirrored_point = np.array([y,x])
        transformed_point = affineTransform(mirrored_point, theta, np.array([position['y'],position['x']]))
        #take first 10 points
        if(len(vectors) <= 19):
            vectors.append(transformed_point[0])
            vectors.append(transformed_point[1])

    return vectors

In [3]:
# Import libraries
import glob
import pandas as pd

# Get CSV files list from a folder
csv_files = glob.glob("training_data/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
df  = pd.concat(df_list, ignore_index=True)

In [4]:
print('Number of rows:', len(df.index))

Number of rows: 584857


In [5]:
df['points'].iloc[583]

"[{'x': 7317.637488608387, 'y': -5271.886918345108, 'z': 0.0}, {'x': 8370.014891390827, 'y': -6324.264321127548, 'z': 0.0}, {'x': 9422.392294173267, 'y': -7376.641723909988, 'z': 0.0}, {'x': 10474.769696955707, 'y': -8429.019126692428, 'z': 0.0}, {'x': 11527.147099738148, 'y': -8429.019126692428, 'z': 0.0}, {'x': 12579.524502520588, 'y': -8429.019126692428, 'z': 0.0}, {'x': 13631.901905303028, 'y': -8429.019126692428, 'z': 0.0}, {'x': 14684.279308085468, 'y': -8429.019126692428, 'z': 0.0}]"

<h1>Transform Data</h1>

In [6]:
import ast
from sklearn import preprocessing

In [7]:
def transformDf(df):
    new_df = pd.DataFrame()
    #string to list/dictionary
    new_df['points'] = df['points'].apply(lambda val: ast.literal_eval(val))
    new_df['position'] = df['position'].apply(lambda val: ast.literal_eval(val))
    new_df['direction'] = df['direction'].apply(lambda val: ast.literal_eval(val))
    
    #apply affine transform
    new_df['points'] = new_df.apply(lambda row: pointsInterface(row['points'], row['direction'], row['position']), axis=1)

    #nicht so schön
    new_df[['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6', 'x7', 'y7', 'x8', 'y8', 'x9', 'y9', 'x10', 'y10']] = pd.DataFrame(new_df.points.tolist(), index= new_df.index)

    #fill NaN with 0
    new_df = new_df.fillna(0)

    #copy throttle, steering, speed
    new_df['speed'] = df['speed']
    new_df['throttle'] = df['throttle']
    new_df['steering'] = df['steering']
    return new_df.drop(['position', 'direction', 'points'], axis=1)

new_df = transformDf(df)

In [8]:
new_df.head()

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y7,x8,y8,x9,y9,x10,y10,speed,throttle,steering
0,-49.136295,9.944135,-49.173788,397.243092,-49.211282,784.542050,338.050183,1171.878500,725.311647,1559.214951,...,2333.887853,1887.096039,2721.224304,2274.357504,3108.560754,2661.656461,3108.598248,0.000000,1.0,0.0
1,-49.136219,9.943898,-49.173716,397.242855,-49.211214,784.541813,338.050246,1171.878268,725.311706,1559.214723,...,2333.887633,1887.096086,2721.224088,2274.357546,3108.560543,2661.656503,3108.598040,0.000000,1.0,0.0
2,-49.136071,9.943613,-49.173575,397.242570,-49.211080,784.541528,338.050373,1171.877990,725.311826,1559.214452,...,2333.887376,1887.096185,2721.223838,2274.357638,3108.560300,2661.656596,3108.597804,0.002367,1.0,0.0
3,-49.135856,9.943370,-49.173369,397.242327,-49.210883,784.541285,338.050562,1171.877755,725.312006,1559.214226,...,2333.887167,1887.096339,2721.223638,2274.357783,3108.560109,2661.656741,3108.597622,0.004025,1.0,0.0
4,-49.135581,9.943207,-49.173104,397.242164,-49.210627,784.541122,338.050808,1171.877602,725.312243,1559.214082,...,2333.887043,1887.096547,2721.223523,2274.357982,3108.560003,2661.656939,3108.597526,0.005277,1.0,0.0


In [ ]:
onepoint = new_df[(new_df['x1'].isnull() == False)]
onepoint = onepoint.sample(frac=1).reset_index(drop=True)
print(onepoint.head())
len(onepoint.index)

In [ ]:
twopoints = new_df[new_df['x2'].isnull() == False]
twopoints = twopoints.sample(frac=1).reset_index(drop=True)
print(twopoints.head())
len(twopoints.index)

In [ ]:
threepoints = new_df[new_df['x3'].isnull() == False]
threepoints = threepoints.sample(frac=1).reset_index(drop=True)
print(threepoints.head())
len(threepoints.index)

In [ ]:
fourpoints = new_df[new_df['x4'].isnull() == False]
fourpoints = fourpoints.sample(frac=1).reset_index(drop=True)
print(fourpoints.head())
len(fourpoints.index)

In [ ]:
fivepoints = new_df[new_df['x10'].isnull() == False]
fivepoints = fivepoints.sample(frac=1).reset_index(drop=True)
print(fivepoints.head())
len(fivepoints.index)

In [ ]:
new_df.head()

In [9]:
#divide data into train/evaluation/test #70%, 15%, 15%
train, validate, test = \
              np.split(new_df.sample(frac=1), 
                       [int(.7*len(new_df)), int(.85*len(new_df))])


In [10]:
train
train.to_csv('train.csv', index=False)

In [11]:
validate
validate.to_csv('validation.csv', index=False)

In [12]:
test
test.to_csv('test.csv', index=False)